# 3. Plots

## 3.1 Set up
Everything that we have done is really nice, but without having a way to visualize the results it may feel odd. It's time to learn about plots.

The default plotting package in Julia is called `Plots.jl` and wraps several plotting environments. If you are not running Binder, uncomment the following lines

In [ ]:
# using Pkg
# Pkg.add("Plots")

For importing it you can run the following cell (**DISCLAIMER**: it may be very slow, but don't despair)

In [ ]:
using Plots 
gr()

In [ ]:
f(x) = x^2
Df(x) = 2*x

# Lets plot the potential 
X = -1:0.01:1
plot(X, f.(X))

## 3.2 1D plots

Ok, it is nice and running. Now we would like to use the code that we developed in notebook 1. One possibility is of course copying it to this notebook, but if we want to use the code in different places this will quickly become cumbersome. 

A clever option is to put the code into a file and then *include* it. This is completely equivalent to running the code in the notebook, so we don't lose anything.

The final version of our `gradient_descent` is saved into the file `GradientDescent.jl`; you may check it. Let us include it and test it:

In [ ]:
include("GradientDescent.jl")

xn, fn = gradient_descent(f, Df, 1., α = 0.1, verbose = true);

Great! Let's plot $f_n$ with respect to the number of iterations

In [ ]:
plot1 = plot(1:length(fn), fn,linewidth=2)
xlabel!("N_iter")
ylabel!("f(x)")
length(fn)

Impresive! Note that we use `xlabel!` because, in some way, we are *modifying* an existing object: the plot! 

Great, but since the curve is so steep we would also like to have some log plot. 

In [ ]:
plot2 = plot(1:length(fn), fn, yscale = :log10, linewidth = 2)
xlabel!("N_iter")
ylabel!("log f(x)")
plot(plot1, plot2, size = (800,300))

So the convergence of the gradient descent is (at least for this example) exponential! Good. We can also visualize it on top of $f(x)$:

In [ ]:
plot(X, f.(X),label="f")
plot!(xn, fn,marker=:star,label="fn",legend=:top)

You may have noticed those strange names with two dots in front, like `:log10` or `:star`. They are *symbols*, and are simply some internal representation of strings. They have more history behind them, but for the moment it may be easier to think of them as some convenient way to representing commonly used strings.

## 3.3 Contour plots

Since we already have a gradient descent that runs for multidimensional input let's do some contourplots:

In [ ]:
g(x) = x[1]^2 + 2*x[2]^2
Dg(x) = [2*x[1], 4*x[2]]

x0 = [0.9, 0.9]
xn, fn = gradient_descent(g, Dg, x0, α = 0.1, maxiter = 20, verbose = true); 
# We can add a semicolon at the end of a statement to mute the output

In [ ]:
G = [g([x, y]) for y in X, x in X]

contourf(X, X, G)
plot!(xn[:,1],xn[:,2],linewidth=2,marker = :star,markerstrokecolor=:lightblue,color=:lightblue,markersize=5,
        aspect_ratio=:equal,legend=:bottomleft,label="xn")
xlims!(-1,1)
ylims!(-1,1)

# 3.4 Gifs and animation

With `Plots.jl`, gifs are trivial to do. Check the following example, in which we animate the still contourplot above:

In [ ]:
@gif for i in 1:length(fn)
    contourf(X, X, G, dpi = 120)  # We changed the color palette for aesthetic reasons
    
    plot!(xn[1:i,1],xn[1:i,2],linewidth=2,marker = :star,markerstrokecolor=:lightblue,color=:lightblue,markersize=5,
        aspect_ratio=:equal,legend=:bottomleft,label="xn")
    xlabel!("x")
    ylabel!("y")
    xlims!(-1,1)
    ylims!(-1,1)
    title!("Gradient descent, It = $(i-1)")
end
# If you are really interested into doing animations you should check the more powerful macro @animate
# You can find some examples here: https://docs.juliaplots.org/latest/animations/

As you see, we have placed a `@gif`, and this *macro* takes care of all the work. Macros take the code that you write and wrap it in some code of their own, to offer some result in a very simple fashion. An even more important macro is `@time`, which is used to measure the time elapsed in running some code:

In [ ]:
@time gradient_descent(f, Df, 1., α = 0.1, verbose = false);

# 3.5 3d and interactive plots

Though `gr` is a neat plots backend, it might come a bit basic for some needs. Another very powerful plot backend is `plotly`, which brings more interactivity:

In [ ]:
plotly() # you may have to do Pkg.add("Plotly")

G = [g([x, y]) for y in X, x in X]
surface(X, X, G, alpha = 0.8,size = (500,600))
xlabel!("x")
ylabel!("y")

plot!(xn[:,1],xn[:,2],fn,color=:lightblue,markersize = 1,marker = :circle,
      markerstrokecolor=:lightblue,linewidth = 3,label="xn",legend=:topleft)
      

# Bonus 1: finetuning animations

For saving the animation that appeared at the beginning of the course I used the macro `@animate`. This is the whole code:

In [ ]:
gr()
anim = @animate for i in 1:length(fn)
    contourf(X, X, G,dpi = 120,aspect_ratio = :equal)
    
    plot!(xn[1:i,1],xn[1:i,2],linewidth=2,marker = :star,markerstrokecolor=:lightblue,color=:lightblue,markersize=5,
        aspect_ratio=:equal,legend=:bottomleft,label="xn")
    xlabel!("x")
    ylabel!("y")
    xlims!(-1,1)
    ylims!(-1,1)
    title!("Contourplot and gradient descent, It = $(i-1)")
end

In [ ]:
gif(anim, "gradient_descent.gif",fps = 2)

# Bonus 2: Advanced performance measuring

An even more powerful macro for measuring performance is `@benchmark`, provided in the package `BenchmarkTools`. Using it you can get more robust measures about the time and memory consumption of a program, as well as detailed statistics. 

In [ ]:
#Pkg.add("BenchmarkTools") # install it if you don't have it already
using BenchmarkTools: @benchmark

@benchmark gradient_descent($f, $Df, 1.) 